In [5]:
! pip install geopandas scikit-learn haversine



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
"""
Objetivo: prever o valor médio das casas (MedHouseValue) usando:
 - variáveis socioeconômicas originais
 - features espaciais criadas a partir de lat/lon
 - validação em blocos geográficos para evitar vazamento
"""
# ---------------- Configuração ----------------
import pandas as pd, geopandas as gpd, numpy as np
from sklearn.datasets import fetch_california_housing
from haversine import haversine_vector, Unit
from sklearn.ensemble import RandomForestRegressor
from spatialcv.model_selection import SpatialKFold   # bloco CV pronto
from sklearn.metrics import mean_absolute_error

# ---------------- 1) Carregar dataset ----------------
cali = fetch_california_housing(as_frame=True)        # baixa ~1 MB:contentReference[oaicite:8]{index=8}
df   = cali.frame
gdf  = gpd.GeoDataFrame(df,
        geometry = gpd.points_from_xy(df.longitude, df.latitude),
        crs="EPSG:4326")                              # CRS geográfico padrão

# ---------------- 2) Feature engineering espacial ----
# Distância ao ponto “centro de Los Angeles” como dummy POI
la_center = (34.05, -118.24)
gdf["dist_LA_m"] = haversine_vector(
        gdf[["latitude","longitude"]].values,
        [la_center], Unit.METERS)

# Densidade populacional aproximada: população / area_km2 de um buffer de 5 km
buffer5 = gdf.to_crs(3857).buffer(5_000)             # projetamos pra métrico
gdf["pop_density_5k"] = (gdf["Population"] /
                         (np.pi*5**2))               # ~pessoas por km²

# ---------------- 3) Preparar X, y --------------------
y = gdf["MedHouseVal"]
X = gdf.drop(columns=["MedHouseVal","geometry"])     # mantemos numéricas

# ---------------- 4) Cross-validation espacial --------
# Divide a Califórnia em 5 blocos lat/lon equilibrados
cv = SpatialKFold(n_splits=5, gdf=gdf, method="block", size=2.5)

maes = []
for train_idx, test_idx in cv.split(gdf):
    model = RandomForestRegressor(n_estimators=400, random_state=42)
    model.fit(X.iloc[train_idx], y.iloc[train_idx])
    preds = model.predict(X.iloc[test_idx])
    maes.append(mean_absolute_error(y.iloc[test_idx], preds))
print(f"MAE médio com CV espacial: US$ {np.mean(maes)*100_000:,.0f}")


IndexError: When not in combination mode, arrays must be of same size. If mode is required, use comb=True as argument.